# GETTING STARTED: Download the Fully Trained Models

We will train our own models in this notebook, but on jupyter notebook it will take too long to train to proper accuracy. As such, we have two fully trained models that you can download at: https://drive.google.com/drive/folders/1oNCkb3baUrRVq9NHZVF1K6af_yfk5BFL?usp=drive_link

Download these two models, and place them within the `trained_ae` folder within this repo.

# REMINDER: This Notebook Uses GPUs

This notebook uses GPUs. There is commented code to switch it to CPUs instead if you like, but for the best performance you want to start an instance with GPUs!

# Autoencoder: Sub-Module of the EEG-GAN Package

The autoencdoer code we are using today is part of our larger package, [EEG-GAN](https://autoresearch.github.io/EEG-GAN/). This package uses generative modelling - specifically, generative adversarial networks - to increase sample sizes by creating synthetic participants. We have shown that doing this enhances classification performance by providing more training samples for classification models. You can read more about this in our [published manuscript](https://escholarship.org/uc/item/9gz8g908). This package may actually be useful here with this data for this challenge; however, it has not been tested using SEEG data nor with so many conditions (i.e., up to 30 video clips) and with so little samples, so we opted not to present this package here. 

With that said, we are actively developing EEG-GAN v2.0, and this future version will contain an embedded autoencoder, which is the code included within this repo. Although this package is in active development, the autoencoder component is stabilized and thoroughly tested to work. 

# Import Modules

In [ ]:
import os
import warnings
import pandas as pd
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm
import scipy
import matplotlib.pyplot as plt

import scipy.signal as ss
import scipy.stats as sst

import torch
from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present
from nn_architecture.ae_networks import TransformerAutoencoder, TransformerDoubleAutoencoder, TransformerFlattenAutoencoder
from helpers.dataloader import Dataloader

pd.set_option('display.max_columns', 5)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Setup Functions

In [ ]:
def notch_data(samples, fs=1024, notch_freq=60.0, quality_factor=20.0, data_prefix='Time'):
    
    #Determine columns containing neural data
    neural_indices = [column_index for column_index, column_name in enumerate(samples.columns) if data_prefix in column_name]

    b_notch, a_notch = ss.iirnotch(notch_freq, quality_factor, fs)
    filtered_signal = ss.filtfilt(b_notch, a_notch, samples.iloc[:, neural_indices[0]:], axis=1)
    samples.iloc[:, neural_indices[0]:] = filtered_signal
    
    return samples

def filter_data(samples, lowcut=0.1, highcut=200, fs=1024, order=5, padding=3, data_prefix='Time'):
    '''
    ...add here
    '''
    padding_datapoints = padding*fs
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = ss.butter(order, [low, high], btype='bandpass')
    
    #Determine columns containing neural data
    neural_indices = [column_index for column_index, column_name in enumerate(samples.columns) if data_prefix in column_name]

    #Filter signal
    filtered_signal = ss.filtfilt(b, a, samples.iloc[:, neural_indices[0]:], padlen=(padding*fs), axis=1)
    samples.iloc[:, neural_indices[0]:] = filtered_signal
    
    return samples

def artifact_rejection(samples, threshold=5, data_prefix='Time'):

    #Determine columns containing neural data
    neural_indices = [column_index for column_index, column_name in enumerate(samples.columns) if data_prefix in column_name]

    #Determine gradients and z-scores
    grad = np.gradient(samples.iloc[:, neural_indices[0]:], axis=1)
    g_zs = sst.zscore(grad, axis=1)
    h_zs = sst.zscore(samples.iloc[:, neural_indices[0]:], axis=1)
    
    #Determine which segments should be removed
    reject = np.logical_or(np.abs(g_zs) > threshold, np.abs(h_zs) > threshold) #Determine all datapoints that violate criteria
    reject = np.any(reject, axis=1)
    
    #Convert to which segments should be kept
    keep = [not r for r in reject]
    
    #Report
    print(f"{np.sum(reject)} of {len(samples)} ({np.round((np.sum(reject)/len(samples))*100)}%) of segments were rejected.")
    
    return samples.loc[keep]

def downsample_data(samples, sampling_rate=1024, downsampled_rate=256, data_prefix='Time'):

    #Determine columns containing neural data
    neural_indices = [column_index for column_index, column_name in enumerate(samples.columns) if data_prefix in column_name]

    #Determine length of samples in seconds
    number_of_seconds = samples.iloc[0][neural_indices[0]:].shape[0]/sampling_rate
    
    #Determine number of datapoints after downsampling
    new_number_of_datapoints = int(number_of_seconds*downsampled_rate)
    
    #Determine new column names
    sr_ratio = sampling_rate/downsampled_rate
    metadata_names = samples.columns[:neural_indices[0]]
    downsampled_time_names = [column_name for column_index, column_name in enumerate(samples.columns[neural_indices[0]:]) if column_index%sr_ratio == 0]
    downsampled_names = list(metadata_names) + list(downsampled_time_names)
    
    #Iterate through samples to downsample
    downsampled_samples = pd.DataFrame(columns=downsampled_names)
    for sample_index in range(len(samples)):
        current_sample = samples.iloc[sample_index]
        current_metadata = current_sample.iloc[:neural_indices[0]]
        current_sample = current_sample.iloc[neural_indices[0]:]
        downsampled_sample = scipy.signal.resample(current_sample, new_number_of_datapoints)
        new_sample = pd.DataFrame([list(current_metadata) + list(downsampled_sample)], columns=downsampled_names)
        downsampled_samples = pd.concat([downsampled_samples, new_sample])

    return downsampled_samples       

def initiate_autoencoder(ae_dict, dataset):

    n_channels = dataset.shape[-1]
    sequence_length = dataset.shape[1] - 1

    if ae_dict['configuration']['target'] == 'channels':
        autoencoder = TransformerAutoencoder(input_dim=n_channels,
                                       output_dim=ae_dict['configuration']['channels_out'],
                                       output_dim_2=sequence_length,
                                       target=TransformerAutoencoder.TARGET_CHANNELS,
                                       hidden_dim=ae_dict['configuration']['hidden_dim'],
                                       num_layers=ae_dict['configuration']['num_layers'],
                                       num_heads=ae_dict['configuration']['num_heads'],).to('cpu')
    elif ae_dict['configuration']['target'] == 'time':
        autoencoder = TransformerAutoencoder(input_dim=sequence_length,
                                       output_dim=ae_dict['configuration']['timeseries_out'],
                                       output_dim_2=n_channels,
                                       target=TransformerAutoencoder.TARGET_TIMESERIES,
                                       hidden_dim=ae_dict['configuration']['hidden_dim'],
                                       num_layers=ae_dict['configuration']['num_layers'],
                                       num_heads=ae_dict['configuration']['num_heads'],).to('cpu')
    elif ae_dict['configuration']['target'] == 'full':
        autoencoder = TransformerDoubleAutoencoder(input_dim=n_channels,
                                             output_dim=ae_dict['configuration']['output_dim'],
                                             output_dim_2=ae_dict['configuration']['output_dim_2'],
                                             sequence_length=sequence_length,
                                             hidden_dim=ae_dict['configuration']['hidden_dim'],
                                             num_layers=ae_dict['configuration']['num_layers'],
                                             num_heads=ae_dict['configuration']['num_heads'],).to('cpu')
    else:
        raise ValueError(f"Encode target '{ae_dict['configuration']['target']}' not recognized, options are 'channels', 'time', or 'full'.")
    consume_prefix_in_state_dict_if_present(ae_dict['model'],'module.')
    autoencoder.load_state_dict(ae_dict['model'])
    autoencoder.device = torch.device('cpu')

    return autoencoder

# Loading and Re-Organizing the Data

The autoencoder needs a very specific data structure to work. The csv files provided for each participant is already in this data stucture, so we will not have to do too much data manipulation in this workshop, which is great.

We use a very simple 2-dimensional data structure where each row is a different trial-level sample for one condition and one electrode. We will direct the autoencoder with column names to guide it in understanding what is metadata and what is neural data. For example, below we will inform the autoencoder that the electrode labels are within the `Electrode` column (i.e., `channel_label=Electrode`) and that the neural data begins with the prefix `Time` (i.e., `kw_timestep=Time`). Behind the scenes, the autoencoder will use the electrode labels to transform the data from a 2D matrix to a 3D matrix and will only include data with the given prefix as the neural data. The autoencoder will ignore any other columns not specified. For example, when looking at the data snippet below, it will ignore the columns `ParticipantID`, `Phase`, and `Condition` because it does not need this information.

| participant_id | Phase | Condition | Electrode | Time0000 | Time0001 | Time0002 | Time0003 | ... |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| e0010GP | Encoding | 1 | HIP1 | 2.28 | -4.44 | -0.98 | -5.67 | ... |
| e0010GP | Encoding | 1 | HIP2 | 11.67 | 0.66 | 1.43 | 11.62 | ... |
| e0010GP | Encoding | 2 | HIP1 | 11.90 | 8.67 | 1.85 | 0.73 | ... |
| e0010GP | Encoding | 2 | HIP2 | 6.73 | 3.63 | 3.80 | 5.63 | ... |
| ... | ... | ... | ... | ... | ... | ... | ... |

## Loading Data

In this workshop, we will be looking through participant data multiple times. Instead of re-loading each dataset over and over again, we might as well load them all once and keep them stored in a variable. We will do this by creating a dictionary where each key is a different participant's dataframe. 

In [ ]:
#First, we designate where the data is
data_path = os.path.abspath('/oscar/data/brainstorm-ws/seeg_data/Memory Task Data/Epilepsy/Monitoring/')

#We will also manually set which participants to include in the autoencoder training
participant_files = [
              f'{data_path}/e0019VQ_preprocessed_data.csv',
              f'{data_path}/e0011XQ_preprocessed_data.csv',
              f'{data_path}/e0022ZG_preprocessed_data.csv',
              f'{data_path}/e0015TJ_preprocessed_data.csv',
              f'{data_path}/e0014VG_preprocessed_data.csv',
              f'{data_path}/e0010GP_preprocessed_data.csv',
              f'{data_path}/e0020JA_preprocessed_data.csv',
              f'{data_path}/e0024DV_preprocessed_data.csv',
              f'{data_path}/e0013LW_preprocessed_data.csv',
              #f'{data_path}/e0017MC_preprocessed_data.csv', #Does not have any electrodes with the HIP naming convention
              #f'{data_path}/e0016YR_preprocessed_data.csv', #Does not have any electrodes with the HIP naming convention
              ]

#We will determine all participant ids 
participant_ids = [participant_file.split('/')[-1].replace('_data.csv','') for participant_file in participant_files]

#Now, let's load all participant data and save them into a dictionary
participant_data = {} #Create empty dictionary
for participant_index, participant_file in enumerate(tqdm(participant_files)):
    participant_data[participant_ids[participant_index]] = pd.read_csv(participant_file, dtype = {'Electrode': str})
                                                                       
#We can report what we created
print(f"Dictionary Keys: {participant_data.keys()}\n")
print('Example dataset:')
print(participant_data[list(participant_data.keys())[0]])

## Electrodes
One other requirement that the autoencoder has is that every participant has the same number of electrodes. This toolbox was designed for EEG analyses, and this is the standard practice in that research---but as we have seen throughout this week, our participants have both a different number of electrodes and electrodes in different locations. What this means is that we are going to have to make some decisions on how many electrodes, and which electrodes, to include for each participant.

Later on, we are going to use an adapted version of Younes Strittmatter's classification notebook, which focuses on hippocampus electrodes (i.e., uses electrodes with the `HIP` acronym), so here we will only focus on these electrodes. Let's cycle through each participant and see which hippocampus electrodes they have.

In [ ]:
#Determine the brain region of interest
brain_region = 'HIP'

#Iterate through all participants
for participant_id in participant_data.keys():
    print(f"Participant: {participant_id}") #Report which participant
    electrodes = [electrode.split('_')[-1] for electrode in participant_data[participant_id]['Electrode'].unique() if brain_region.lower() in electrode.lower()] #Determine corresponding electrodes
    print(f"Electrodes ({len(electrodes)}):\n{electrodes}\n") #Report found electrodes

You will notice that the participants have quite different electrodes within the hippocamps. This could mean that participants have electrodes in different locations but in some cases it could also simply indicate that there are different naming conventions per participant. Ideally, we would be able to find a number of electrodes that are completely consistent across participants but this may not be possible with the current dataset due to the nature of collecting SEEG data. 

So, instead of selecting common electrodes, we will randomly select a set of electrodes for each participant and hope that they are similar enough to consider consistent across participants. The classification code that Younes presented has us averaging across these electrodes anyhow, so this seems like a safe enough process here. 

The participants also have a different number of electrodes within the hippocampus. As the autoencoder needs the same number of electrodes for each participant, we need to select a number of electrodes equal to or less than the lower end of this range.

You could select which electrodes to include manually if you would like, but instead here we will randomly select hippocampus electrodes. We will set a seed in our `numpy` random function first so that we end up with the same electrodes every time we run this code. 

In [ ]:
#Set numpy seed
np.random.seed(42)

#Determine the brain region of interest
brain_region = 'HIP'
number_of_electrodes = 20

#Iterate through each participant and collect 12 electrodes per participant 
selected_electrodes = {} #Setup a variable to collect the selected electrodes
for participant_id in participant_data.keys():
    electrodes = [electrode for electrode in participant_data[participant_id]['Electrode'].unique() if brain_region.lower() in electrode.lower()] #Determine corresponding electrodes
    if electrodes:
        electrodes = np.random.choice(electrodes, number_of_electrodes, replace=False) #Randomly select a number of electrodes equal to the variable number_of_electrodes
    selected_electrodes[participant_id] = electrodes #Save selected electrodes as a list of lists to use later
    
#Print as dataframe
pd.set_option('display.max_columns', len(participant_files)) #Show more columns just for a moment
print(pd.DataFrame(np.array([selected_electrodes[key] for key in selected_electrodes.keys()]).T, columns = [participant_file.split('/')[-1].replace('_data.csv','') for participant_file in participant_files]))
pd.set_option('display.max_columns', 5) #Reset to showing 5 columns max

## Combining Data Files

Another requirement the autoencoder has is that all of the data is within the same csv file. Above we have created a list of participants to include and we have determined which electrodes to use per participant, so now we will extract the participant data for each participant and add it all to a single dataframe and then save this dataframe in our current repo's `data` sub-directory. 

The autoencoder should only be trained with the data that you will use to train your classification model. In the classification code, we will train our model on the `Encoding` and `SameDayRecall` phases and validate it's performance on the `NextDayRecall` phase.

Furthermore, the autoencoder model can get quite big with large datasets, and having 5 seconds at 1024Hz (i.e., 5120 datapoints) is quite large. Large data and models may fail if they fill your memory. So, we will here condense by downsampling the data using the `downsample_data` function. In this workshop we will reduce the sampling rate from 1024Hz to 128Hz.

WARNING: Sharp wave ripples (SWR) are in the 80 - 100Hz range and the frequencies you can extract from data equal half of the sampling rate (this is called the Nyquist Limit). So, if we downsample to 128Hz, we can only extract frequencies up to 64Hz, cutting out all of these SWR frequencies. If we want to keep these SWR frequencies, it would be better to downsample to 256Hz, which would allow us to extract frequencies up to 128Hz.

In [ ]:
def create_data_files(participant_data, selected_electrodes, phases=['Encoding', 'SameDayRecall'], conditions = [1, 2], save_name='data/seeg_ae_training_data.csv'):
    #We will first create a dataframe to store all of our participant data in
    times = [f"Time{str(datapoint).zfill(4)}" for datapoint in range(5120)]
    ae_data = pd.DataFrame(columns=['Participant_ID','Phase','Condition','Electrode','Error_Position','Error_Color']+times)
    
    #Iterate through each participant, filtering the data to the specified phases and electrodes, and store it into the master variable
    for participant_id in tqdm(participant_data.keys()):
        phase_indices = participant_data[participant_id]['Phase'].isin(phases) #Phase indices
        electrode_indices = participant_data[participant_id]['Electrode'].isin(selected_electrodes[participant_id]) #Electrode indices
        conditions_indices = participant_data[participant_id]['Condition'].isin(conditions)
        p_data = participant_data[participant_id][(phase_indices) & (electrode_indices) & (conditions_indices)] #Extract participant data
        
        #We also need to rename all of the electrodes so that there is consistency of electrode labels across participants
        for electrode_index, selected_electrode in enumerate(selected_electrodes[participant_id]):
            p_data = p_data.replace({selected_electrode: electrode_index+1})

        for phase_index, selected_phase in enumerate(phases):
            p_data = p_data.replace({selected_phase: phase_index+1})
            
        #Pre-Process Data
        if p_data.shape[0] > 0: #Check to see if data contains variables of interest
            #p_data = notch_data(p_data, fs=1024, notch_freq=60.0, quality_factor=20.0, data_prefix='Time')
            #p_data = filter_data(p_data, lowcut=0.1, highcut=120, fs=1024, order=5, padding=3, data_prefix='Time') #Note: You should really only filter the continuous data, and not segmented data like we are doing here because the latter will introduct 'edge artifacts'
            #p_data = artifact_rejection(p_data, threshold=10, data_prefix='Time') #Note: Artifact rejection might be an issue here, as it can cause some electrodes to have no data. The autoencoder requires all specified electrodes to have data. So, you might rather introduce artifact rejection after reconstructing the data via the autoencoder.
            p_data = downsample_data(p_data, sampling_rate=1024, downsampled_rate=128, data_prefix='Time')

            #Add data to the master variable
            ae_data = pd.concat([ae_data, p_data])

    #Sort data
    ae_data = ae_data.sort_values(by=['Phase', 'Condition', 'Participant_ID', 'Electrode'])
    
    #Save data to csv
    ae_data = ae_data.dropna(how="all", axis=1) #Removes columns that are empty due to downsampling
    ae_data.to_csv(save_name, index=False)

In [ ]:
print('Creating training data...')
create_data_files(participant_data, selected_electrodes, phases=['Encoding', 'SameDayRecall'], conditions = [1,2], save_name='data/seeg_ae_training_data.csv')

print('Creating validation data...')
create_data_files(participant_data, selected_electrodes, phases=['NextDayRecall'], conditions = [1,2], save_name='data/seeg_ae_validation_data.csv')

We can see the file is now within our directory. As this file is now ready and in our directory, we are ready to train the autoencoder!

In [ ]:
!dir data

# Autoencoder

The autoencoder is designed to be used within the command line interface (CLI). What this means is that we do not use functions, such as `train_autoencoder(training_data)` as is common with Python. Insteead we are going to run commands such as `python autoencoder_training_main.py ...` where `...` are our input parameters. If you are not familiar with CLI programming, not to worry as we will show you how to do so here. 

In jupyter notebooks, if you begin a line of code with `!` it will run via the CLI. Our Autoencoder has a `help` parameter so let's see how that works here:

In [ ]:
!python autoencoder_training_main.py help

## Parameters

The help menu above provides a full list of adjustable parameters, but most often only a handfull of these parameters are going to be tweaked, so let's outline these here.

`path_dataset`: This is the path to our dataset<br>
`save_name`: Path to save the file to (note, this seems not to function on Oscar for some reason)<br><br>

`kw_timestep`: The column name prefix of your neural data<br>
`channel_label`: The column name of your electrode labels<br><br>

`target`: The dimension to encode data, we can encode the timeseries (`time`), the electrodes (`channel`), or both (`full`)<br>
`timeseries_out`: The encoded size for electrodes. Only applies if `target` equals `channel` or `full`<br>
`channels_out`: The encoded size for electrodes. Only applies if `target` equals `channel` or `full`<br><br>

`n_epochs`: The number of epochs to train the autoencoder<br>
`ddp`: Whether to activate distributed data parallel training. This means that training will occurs across all GPUs. This does not directly quicken training but makes it more efficient, thus needing less epochs for the same results 

# Autoencoder (Sample Data)

We will train an autoencoder on the SEEG data momentarily, but for a quick proof of concept let's apply it to our sample data (`ganTrialElectrodeERP_p50_e8_len100.csv`).

This data contains `8 electrodes` and `100 datapoints`. The data actually spans 1200ms and was originally recorded at `512Hz` but we have downsampled it to 100 datapoints, which gives it an effect sampling rate of `~83Hz`.

Here, we will use the autoencoder to reduce both the electrode and time dimension in half. This means that we will have reduced the data to `1/4` of it's size.<br>
`Time`: 100 datapoints -> 50 datapoints<br>
`Electrodes`: 8 electrodes -> 4 electrodes<br>

We will train for 10 epochs just to quicken the process or else it would take too long to run in this workshop. We will also use ddp training here.<br>

Let's define these parameters now:

In [ ]:
path_dataset = 'data/ganTrialElectrodeERP_p50_e8_len100.csv'
save_name = 'trained_ae/sample_data_autoencoder.pt'

kw_timestep = 'Time'
channel_label = 'Electrode'

target = 'full'
timeseries_out = 50
channels_out = 4

n_epochs = 10

Now that we have defined our parameters, we can run the training code via the command line interface by starting our command with `!`. Note that the following script has the ddp parameter in it. If you do not have GPUs this will crash. Just delete this parameter and you can train on CPUs.

In [ ]:
#Using GPU:
!python autoencoder_training_main.py ddp path_dataset={path_dataset} save_name={save_name} target={target} kw_timestep={kw_timestep} channel_label={channel_label} channels_out={channels_out} timeseries_out={timeseries_out} n_epochs={n_epochs}

#Using CPU
#!python autoencoder_training_main.py path_dataset={path_dataset} save_name={save_name} target={target} kw_timestep={kw_timestep} channel_label={channel_label} channels_out={channels_out} timeseries_out={timeseries_out} n_epochs={n_epochs}

Above you will see what the autoencoder was saved as and you can see the new autoencoder saved into the `trained_ae` folder. Again, the `save_name` parameter should have saved the autoencoder with the name specified, but for some reason this does not work on Oscar.

In [ ]:
!dir trained_ae

# Evaluating the Autoencoder (Sample Data)

Now that the autoencoder is trained, it's important to see if it worked. We can do this by loading data, and running that data through the autoencoder and see how closely the original data and the autoencoder-reconstructed data match. 

First, let's load the autoencoder and view the configuration of the autoencoder.

In [ ]:
#Determine paths to data and autoencoder
#ae_checkpoint = 'trained_ae/ae_ddp_500ep_20240118_123640.pt'
ae_checkpoint = 'trained_ae/fully_trained_sample_data_autoencoder.pt'

#Load the autoencoder
ae_dict = torch.load(ae_checkpoint, map_location=torch.device('cpu'))

#Report
for key in ae_dict['configuration'].keys():
    if key != 'dataloader' and key != 'history':
        print(f"{key}: {ae_dict['configuration'][key]}")

Now, we can load the data and see how well the autoencoder can reconstruct the data after reducing it to the given dimensions.

In [ ]:
#Load and normalize the data to be within [0,1] (as the autoencoder uses normalized data)
data_checkpoint = 'data/ganTrialElectrodeERP_p50_e8_len100.csv'
dataloader = Dataloader(data_checkpoint, col_label='Condition', channel_label='Electrode')
dataset = dataloader.get_data()
def norm(data):
    return (data-np.min(data)) / (np.max(data) - np.min(data))
dataset = norm(dataset.detach().numpy())

#Initiate autoencoder
autoencoder = initiate_autoencoder(ae_dict, dataset)

#Plot 5 random samples to see reconstruction
plt.rcParams['figure.figsize'] = [10, 10] #Set figure size
fig, axs = plt.subplots(5,1) #Create figure
for i in range(5):
    sample = np.random.choice(len(dataset), 1) #Randomly determine which sample to plot
    data = dataset[sample,1:,:] #Retrieve the sample to plot
    axs[i].plot(data[0,:,0], alpha=.5, label='Original') #Plot data
    axs[i].plot(autoencoder.decode(autoencoder.encode(torch.from_numpy(data)))[0,:,0].detach().numpy(), alpha=.5, label='Reconstructed') #Plot reconstructed data
    axs[i].legend() #Plot legend
plt.show() #Show plot

# Autoencoder (SEEG Data)

Now let's train the autoencoder on the SEEG data that we created at the beginning of this workshop (`seeg_ae_training_data.csv`).

This data contains the `12 electrodes` that we chose. Furthermore, we downsampled the data from `1024Hz` to `128Hz`, and contains 5 seconds of data, so we have `128*5 = 640` datapoints.

Here, we will use the autoencoder to reduce both the electrode and time dimension in half. This means that we will have reduced the data to `1/4` of it's size.<br>
`Time`: 640 datapoints -> 320 datapoints<br>
`Electrodes`: 20 electrodes -> 10 electrodes<br>

We will train for 1000 epochs just to quicken the process or else it would take too long to run in this workshop. We will also use ddp training here.<br>

Let's define these parameters now:

In [ ]:
path_dataset = 'data/seeg_ae_training_data.csv'
save_name = 'trained_ae/seeg_autoencoder.pt'

kw_timestep = 'Time'
channel_label = 'Electrode'

target = 'full'
timeseries_out = 320
channels_out = 10

n_epochs = 1000

In [ ]:
#Using GPU:
!python autoencoder_training_main.py ddp path_dataset={path_dataset} save_name={save_name} target={target} kw_timestep={kw_timestep} channel_label={channel_label} channels_out={channels_out} timeseries_out={timeseries_out} n_epochs={n_epochs} batch_size=16

#Using CPU:
#!python autoencoder_training_main.py path_dataset={path_dataset} save_name={save_name} target={target} kw_timestep={kw_timestep} channel_label={channel_label} channels_out={channels_out} timeseries_out={timeseries_out} n_epochs={n_epochs} batch_size=16

# Evaluating the Autoencoder (SEEG Data)

Now that the autoencoder is trained, it's important to see if it worked. We can do this by loading data, and running that data through the autoencoder and see how closely the original data and the autoencoder-reconstructed data match. 

First, let's load the autoencoder and view the configuration of the autoencoder.

In [ ]:
#Determine paths to data and autoencoder
#ae_checkpoint = 'trained_ae/ae_ddp_5000ep_20240119_094157.pt' #<------Your trained model save name may look something like this
ae_checkpoint = 'trained_ae/fully_trained_seeg_autoencoder.pt'

#Load the autoencoder
ae_dict = torch.load(ae_checkpoint, map_location=torch.device('cpu'))

#Report
for key in ae_dict['configuration'].keys():
    if key != 'dataloader' and key != 'history':
        print(f"{key}: {ae_dict['configuration'][key]}")

Now, we can load the data and see how well the autoencoder can reconstruct the data after reducing it to the given dimensions.

In [ ]:
#Load and normalize the data to be within [0,1] (as the autoencoder uses normalized data)
data_checkpoint = 'data/seeg_ae_training_data.csv'
dataloader = Dataloader(data_checkpoint, col_label='Condition', channel_label='Electrode')
dataset = dataloader.get_data()
def norm(data):
    return (data-np.min(data)) / (np.max(data) - np.min(data))
dataset = norm(dataset.detach().numpy())

#Initiate autoencoder
autoencoder = initiate_autoencoder(ae_dict, dataset)

#Plot 5 random samples to see reconstruction
plt.rcParams['figure.figsize'] = [10, 10] #Set figure size
fig, axs = plt.subplots(5,1) #Create figure
for i in range(5):
    sample = np.random.choice(len(dataset), 1) #Randomly determine which sample to plot
    data = dataset[sample,1:,:] #Retrieve the sample to plot
    axs[i].plot(data[0,:,0], alpha=.5, label='Original') #Plot data
    axs[i].plot(autoencoder.decode(autoencoder.encode(torch.from_numpy(data)))[0,:,0].detach().numpy(), alpha=.5, label='Reconstructed') #Plot reconstructed data
    axs[i].legend() #Plot legend
plt.show() #Show plot